# Sesi Yazıya Dönüştürme

In [1]:
pip install pydub SpeechRecognition wave vosk ffprobe ffmpeg python-docx


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Install


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\oguzo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
# m4a dosyasını soldaki dosyalar kısmına yükleyin ve bu kod sayesinde wav formatına çevirin
from pydub import AudioSegment
sound = AudioSegment.from_file("dosya_adı", format="m4a")
sound.export("dosya_adı.wav", format="wav")

FileNotFoundError: [Errno 2] No such file or directory: 'dosya_adı'

In [ ]:
# Eğer büyük bir dosya varsa transkripte çevirirken sorun yaratmasın diye parçalara bölebiliriz
from pydub import AudioSegment
import math

# WAV dosyasını yükleyin
wav_file = "dosya_adı.wav"
sound = AudioSegment.from_file(wav_file)

# 52 parçaya ayırıyor
parca_suresi = len(sound) / 52
for i in range(52):
    start_time = i * parca_suresi  # Başlangıç süresi (milisaniye cinsinden)
    end_time = (i + 1) * parca_suresi  # Bitiş süresi

    # Her parçayı kesip kaydedin
    parca = sound[start_time:end_time]

    # Parçayı sırayla numaralandırarak kaydedin
    parca.export(f"parca_{i+1}.wav", format="wav")

print("WAV dosyası başarıyla 52 parçaya bölündü.")  # Daha rahat işleyebilmek için



In [ ]:
# 52 parçaya ayırıp size word dosyası veren kod
from pydub import AudioSegment
import speech_recognition as sr
from docx import Document

# WAV dosyasını yükleyin
wav_file = "parca_1.wav"
sound = AudioSegment.from_file(wav_file)

# Dosyayı 52 parçaya bölmek için her parçanın süresini hesaplayın
parca_suresi = len(sound) / 52  # Her parçanın süresi (milisaniye cinsinden)

# SpeechRecognition için hazırlık
recognizer = sr.Recognizer()

# Word belgesi oluşturun
document = Document()

# Her bir parçayı işle ve metni Word dosyasına ekle
for i in range(52):
    start_time = i * parca_suresi  # Başlangıç süresi
    end_time = (i + 1) * parca_suresi  # Bitiş süresi

    # Parçayı kes
    parca = sound[start_time:end_time]

    # Parçayı geçici bir dosyaya kaydet (SpeechRecognition için)
    parca.export(f"parca_{i+1}.wav", format="wav")

    # Parçayı metne dönüştür
    with sr.AudioFile(f"parca_{i+1}.wav") as source:
        audio_data = recognizer.record(source)
        try:
            # Türkçe dilinde çözümle
            metin = recognizer.recognize_google(audio_data, language="tr-TR")
        except sr.UnknownValueError:
            metin = f"Parça {i+1} çözümlenemedi."
        except sr.RequestError as e:
            metin = f"Hizmet hatası: {e}"

    # Her bir metni Word belgesine ekle
    document.add_paragraph(f"Parça {i+1}:")
    document.add_paragraph(metin)
    document.add_paragraph("\n")  # Boşluk bırak

    print(f"Parça {i+1} tamamlandı.")

# Word dosyasını kaydet
document.save("ses_dosyasi_transkripti.docx")

print("Transkript başarıyla Word dosyasına kaydedildi.")
